In [8]:
pip install tensorflow pandas scikit-learn numpy


Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df = pd.read_csv('/Users/rajbehera/Documents/projects/weather_data.csv', delimiter=';')

# Parse dates if your dataset includes a date column (assuming the column is named 'date')
df['date'] = pd.to_datetime(df['date'])

# Choose a target variable, for example, maximum temperature ('tmax')
target_variable = 'tmax'

# Normalize the target variable to scale the data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
df[target_variable] = scaler.fit_transform(df[[target_variable]].values.astype(float))




In [11]:
import numpy as np

def create_sequences(data, sequence_length):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        xs.append(data[i:(i + sequence_length)])
        ys.append(data[i + sequence_length])
    return np.array(xs), np.array(ys)

sequence_length = 14  # Example: Use the last 14 days to predict the next day
X, y = create_sequences(df[target_variable].values, sequence_length)


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')


In [16]:
# Note: Reshape the data for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

model.fit(X_train, y_train, epochs=10000, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0196 - val_loss: 0.0185
Epoch 2/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0194 - val_loss: 0.0191
Epoch 3/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0195 - val_loss: 0.0188
Epoch 4/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0196 - val_loss: 0.0180
Epoch 5/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0192 - val_loss: 0.0182
Epoch 6/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0196 - val_loss: 0.0205
Epoch 7/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0196 - val_loss: 0.0184
Epoch 8/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0190 - val_loss: 0.0185
Epoch 9/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0193 - val_loss: 0.0192
Epoch 10/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0190 - val_lo

In [17]:
# Make predictions
predictions = model.predict(X_test)

# Inverse transform predictions
predictions = scaler.inverse_transform(predictions)

# Evaluate the model's performance (you can use metrics such as Mean Squared Error)


13/13 [==============================] - 0s 1ms/step


In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [19]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, predictions)

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, predictions)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Absolute Error: {mae}")


Mean Squared Error: 5290.43022242316
Root Mean Squared Error: 72.73534369495452
Mean Absolute Error: 71.29906685341406
